In [1]:
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import balanced_accuracy_score,roc_auc_score,make_scorer
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix#create a confusion matrix
from sklearn.metrics import plot_confusion_matrix#plot a confusion matrix

In [2]:
data= pd.read_csv('C:/Users/meetm/Desktop/marksheet2.csv')
data.head()

,id,Name,Gender,Age,Section,Science,English,History,Maths,total,result
0,1,Bronnie,Female,13,C,21.0,81,62.0,49.0,213,1
1,2,Lemmie,Male,15,B,29.0,12,17.0,40.0,98,0
2,3,Danya,Female,14,C,12.0,87,16.0,96.0,211,1
3,4,Denna,Female,14,B,15.0,98,82.0,33.0,228,1
4,5,Jocelin,Male,14,A,43.0,6,3.0,21.0,73,0


In [3]:
data['Gender'].unique()

array(['Female', 'Male'], dtype=object)

In [4]:
data['Section'].unique()

array(['C', 'B', 'A'], dtype=object)

In [5]:
data['Age'].unique()

array([13, 15, 14], dtype=int64)

In [6]:
data

,id,Name,Gender,Age,Section,Science,English,History,Maths,total,result
0,1,Bronnie,Female,13,C,21.0,81,62.0,49.0,213,1
1,2,Lemmie,Male,15,B,29.0,12,17.0,40.0,98,0
2,3,Danya,Female,14,C,12.0,87,16.0,96.0,211,1
3,4,Denna,Female,14,B,15.0,98,82.0,33.0,228,1
4,5,Jocelin,Male,14,A,43.0,6,3.0,21.0,73,0
...,...,...,...,...,...,...,...,...,...,...,...
245,246,Nickie,Male,13,C,28.0,15,25.0,10.0,78,0
246,247,Rog,Female,13,B,1.0,4,68.0,65.0,138,0
247,248,Kaia,Male,15,B,93.0,48,82.0,44.0,267,1
248,249,Anni,Female,14,B,35.0,73,66.0,59.0,233,1


In [7]:
data.drop(['id'],axis=1)

,Name,Gender,Age,Section,Science,English,History,Maths,total,result
0,Bronnie,Female,13,C,21.0,81,62.0,49.0,213,1
1,Lemmie,Male,15,B,29.0,12,17.0,40.0,98,0
2,Danya,Female,14,C,12.0,87,16.0,96.0,211,1
3,Denna,Female,14,B,15.0,98,82.0,33.0,228,1
4,Jocelin,Male,14,A,43.0,6,3.0,21.0,73,0
...,...,...,...,...,...,...,...,...,...,...
245,Nickie,Male,13,C,28.0,15,25.0,10.0,78,0
246,Rog,Female,13,B,1.0,4,68.0,65.0,138,0
247,Kaia,Male,15,B,93.0,48,82.0,44.0,267,1
248,Anni,Female,14,B,35.0,73,66.0,59.0,233,1


In [8]:
data.dtypes

id           int64
Name        object
Gender      object
Age          int64
Section     object
Science    float64
English      int64
History    float64
Maths      float64
total        int64
result       int64
dtype: object

In [9]:
len(data.loc[data['English']==''])


0

In [10]:
data.loc[data['English']=='0']

,id,Name,Gender,Age,Section,Science,English,History,Maths,total,result


In [11]:
data.isnull().sum(axis = 0)

id         0
Name       0
Gender     0
Age        0
Section    0
Science    3
English    0
History    1
Maths      1
total      0
result     0
dtype: int64

In [12]:
data[data['English'].isna()]

,id,Name,Gender,Age,Section,Science,English,History,Maths,total,result


In [13]:
data.isnull().sum(axis=0).sort_values(ascending = False)

Science    3
History    1
Maths      1
id         0
Name       0
Gender     0
Age        0
Section    0
English    0
total      0
result     0
dtype: int64

In [14]:
data[data['English'].isnull()]

,id,Name,Gender,Age,Section,Science,English,History,Maths,total,result


In [15]:
data[data['Science'].isnull()]

,id,Name,Gender,Age,Section,Science,English,History,Maths,total,result
69,70,Jessamine,Female,14,B,NaN,71,35.0,47.0,153,0
71,72,Jaymie,Male,14,C,NaN,59,60.0,NaN,119,0
73,74,Bethena,Male,13,B,NaN,89,83.0,64.0,236,1


In [16]:
data[data['History'].isnull()]

,id,Name,Gender,Age,Section,Science,English,History,Maths,total,result
68,69,Nil,Female,15,A,84.0,97,NaN,96.0,277,1


In [17]:
data[data.isna().any(axis=1)]

,id,Name,Gender,Age,Section,Science,English,History,Maths,total,result
68,69,Nil,Female,15,A,84.0,97,NaN,96.0,277,1
69,70,Jessamine,Female,14,B,NaN,71,35.0,47.0,153,0
71,72,Jaymie,Male,14,C,NaN,59,60.0,NaN,119,0
73,74,Bethena,Male,13,B,NaN,89,83.0,64.0,236,1


In [18]:
data['English']=pd.to_numeric(data['English'])
data['English']

0      81
1      12
2      87
3      98
4       6
       ..
245    15
246     4
247    48
248    73
249     8
Name: English, Length: 250, dtype: int64

In [19]:
data.dtypes

id           int64
Name        object
Gender      object
Age          int64
Section     object
Science    float64
English      int64
History    float64
Maths      float64
total        int64
result       int64
dtype: object

In [20]:
#data= data.replace(to_replace = np.nan, value ='_')

In [21]:
data

,id,Name,Gender,Age,Section,Science,English,History,Maths,total,result
0,1,Bronnie,Female,13,C,21.0,81,62.0,49.0,213,1
1,2,Lemmie,Male,15,B,29.0,12,17.0,40.0,98,0
2,3,Danya,Female,14,C,12.0,87,16.0,96.0,211,1
3,4,Denna,Female,14,B,15.0,98,82.0,33.0,228,1
4,5,Jocelin,Male,14,A,43.0,6,3.0,21.0,73,0
...,...,...,...,...,...,...,...,...,...,...,...
245,246,Nickie,Male,13,C,28.0,15,25.0,10.0,78,0
246,247,Rog,Female,13,B,1.0,4,68.0,65.0,138,0
247,248,Kaia,Male,15,B,93.0,48,82.0,44.0,267,1
248,249,Anni,Female,14,B,35.0,73,66.0,59.0,233,1


In [1]:
X=data.drop('result',axis=1).copy()
X

NameError: name 'data' is not defined

In [46]:
X=X.drop(['Name','id','Gender'],axis=1)

In [47]:
y=data['result'].copy()
y.head()

0    1
1    0
2    1
3    1
4    0
Name: result, dtype: int64

In [48]:
X.dtypes

Age          int64
Section     object
Science    float64
English      int64
History    float64
Maths      float64
total        int64
dtype: object

In [49]:
#get dummies data by spliting the data (it is one hot coding )
#here we have splitted value of age
pd.get_dummies(X,columns=['Age']).head()

,Section,Science,English,History,Maths,total,Age_13,Age_14,Age_15
0,C,21.0,81,62.0,49.0,213,1,0,0
1,B,29.0,12,17.0,40.0,98,0,0,1
2,C,12.0,87,16.0,96.0,211,0,1,0
3,B,15.0,98,82.0,33.0,228,0,1,0
4,A,43.0,6,3.0,21.0,73,0,1,0


In [50]:
X_encoded=pd.get_dummies(X,columns=['Section'])

In [51]:
X_encoded.head()

,Age,Science,English,History,Maths,total,Section_A,Section_B,Section_C
0,13,21.0,81,62.0,49.0,213,0,0,1
1,15,29.0,12,17.0,40.0,98,0,1,0
2,14,12.0,87,16.0,96.0,211,0,0,1
3,14,15.0,98,82.0,33.0,228,0,1,0
4,14,43.0,6,3.0,21.0,73,1,0,0


In [52]:
y.unique()

array([1, 0], dtype=int64)

In [53]:
sum(y)/len(y)

0.76

In [54]:
#above we got is an average of marks student scored in english


In [55]:
X_train,X_test,y_train,y_test= train_test_split(X_encoded,y,random_state=42,stratify=y)

In [56]:
sum(y_train)/len(y_train)

0.7593582887700535

In [57]:
sum(y_test)/len(y_test)

0.7619047619047619

In [60]:
clf_xgb=xgb.XGBClassifier(objective='binary:logistic',missing=None,seed=42)

In [59]:
X

,Age,Section,Science,English,History,Maths,total
0,13,C,21.0,81,62.0,49.0,213
1,15,B,29.0,12,17.0,40.0,98
2,14,C,12.0,87,16.0,96.0,211
3,14,B,15.0,98,82.0,33.0,228
4,14,A,43.0,6,3.0,21.0,73
...,...,...,...,...,...,...,...
245,13,C,28.0,15,25.0,10.0,78
246,13,B,1.0,4,68.0,65.0,138
247,15,B,93.0,48,82.0,44.0,267
248,14,B,35.0,73,66.0,59.0,233


In [37]:
clf_xgb.fit(X_train,y_train,verbose=True,early_stopping_rounds=10,eval_metric='aucpr',eval_set=[X_test,y_test])

C:\Users\meetm\AppData\Roaming\Python\Python39\site-packages\xgboost\sklearn.py:793: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(
C:\Users\meetm\AppData\Roaming\Python\Python39\site-packages\xgboost\sklearn.py:793: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


ValueError: too many values to unpack (expected 2)